In [3]:
import torch
from ultralytics import YOLO

model_path = '~/shifumi_trained.pt'

model = YOLO('yolov9c.pt')
model.eval()

⚠️ Download failure, retrying 1/3 https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov9c.pt...


######################################################################## 100.0%


Ultralytics YOLOv8.2.45 🚀 Python-3.12.0 torch-2.3.1 CPU (Apple M1)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

RuntimeError: Dataset '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' error ❌ '/usr/src/ultralytics/ultralytics/cfg/datasets/coco.yaml' does not exist

In [ ]:
import cv2

# Initialiser la capture vidéo (0 pour la caméra par défaut)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Erreur: Impossible d'ouvrir la caméra.")
    exit()

# Définir les dimensions de la fenêtre
width = 640
height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Erreur: Impossible de lire la frame.")
        break

    # Convertir la frame en RGB
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Faire une prédiction
    results = model(img_rgb)

    # Traiter les résultats
    labels, coords = results.xyxyn[0][:, -1], results.xyxyn[0][:, :-1]

    # Parcourir les résultats et dessiner les boîtes englobantes
    for i in range(len(labels)):
        row = coords[i]
        if row[4] >= 0.5:  # Confiance minimale de 0.5
            x1, y1, x2, y2 = int(row[0] * width), int(row[1] * height), int(row[2] * width), int(row[3] * height)
            bgr = (0, 255, 0)  # Couleur des boîtes englobantes
            cv2.rectangle(frame, (x1, y1), (x2, y2), bgr, 2)
            cv2.putText(frame, f'{model.names[int(labels[i])]} {row[4]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, bgr, 2)

    # Afficher la frame avec les boîtes englobantes
    cv2.imshow('YOLOv9 Detection', frame)

    # Quitter avec la touche 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libérer la capture vidéo et fermer les fenêtres
cap.release()
cv2.destroyAllWindows()
